In [1]:
!pip install condacolab
import condacolab
condacolab.install_mambaforge()

Mambaforge has been sunset. It is now identical to Miniforge. Installing Miniforge...


⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:18
🔁 Restarting kernel...


In [2]:
!mamba install -y -c conda-forge openmmforcefields pdbfixer


Looking for: ['openmmforcefields', 'pdbfixer']

[+] 0.0s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.1s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.2s
conda-forge/linux-64   2%
conda-forge/noarch     1%[+] 0.3s
conda-forge/linux-64   4%
conda-forge/noarch     2%[+] 0.4s
conda-forge/linux-64   6%
conda-forge/noarch     4%[+] 0.5s
conda-forge/linux-64   8%
conda-forge/noarch    11%[+] 0.6s
conda-forge/linux-64   9%
conda-forge/noarch    13%[+] 0.7s
conda-forge/linux-64  11%
conda-forge/noarch    17%[+] 0.8s
conda-forge/linux-64  13%
conda-forge/noarch    19%[+] 0.9s
conda-forge/linux-64  15%
conda-forge/noarch    24%[+] 1.0s
conda-forge/linux-64  15%
conda-forge/noarch    26%[+] 1.1s
conda-forge/linux-64  17%
conda-forge/noarch    30%[+] 1.2s
conda-forge/linux-64  18%
conda-forge/noarch    32%[+] 1.3s
conda-forge/linux-64  19%
conda-forge/noarch    34%[+] 1.4s
conda-forge/linux-64  20%
conda-forge/noarch    36%[+] 1.5s
conda-forge/linux-64  21%
conda-forge/noar

In [1]:
from openff.toolkit.topology import Molecule
from openff.toolkit import Topology as offTopology
from openff.units.openmm import to_openmm as offquantity_to_openmm

from openmmforcefields.generators import SMIRNOFFTemplateGenerator
# from openmm.app import ForceField, Modeller, PDBFile
# from openmm.unit import *
from openmm import *
from openmm.app import *
from openmm.unit import *

In [2]:
## Set up platform
print("Setting up the platform...")
# platform = Platform.getPlatformByName('CUDA')
platform = Platform.getPlatformByName('CPU')
# platform.setPropertyDefaultValue('Precision', 'mixed')

Setting up the platform...


In [3]:
## Create an OpenFF Molecule object from SMILES
# molecule = Molecule.from_smiles('CC(=O)NC3C(O)OC(CO)C(OC2OC(CO)C(O)C(OC1(C(=O)O)CC(O)C(NC(C)=O)C(C(O)C(O)CO)O1)C2O)C3O', allow_undefined_stereo=True)

molecule = Molecule.from_file('epi1846961__4k63_i_model_mol_withH.sdf')

In [4]:
molecule.assign_partial_charges(partial_charge_method="mmff94")

In [5]:
## Create the SMIRNOFF template generator
smirnoff = SMIRNOFFTemplateGenerator(molecules=molecule)

In [6]:
## Create an OpenMM ForceField object
# forcefield = ForceField('amber/protein.ff14SB.xml', 'amber/tip3p_standard.xml', 'amber/tip3p_HFE_multivalent.xml')
# forcefield = forcefield = ForceField('charmm36.xml', 'charmm36/water.xml')
forcefield = ForceField('amber/protein.ff14SB.xml', 'amber/tip3p_standard.xml')

In [7]:
## Register the SMIRNOFF template generator
forcefield.registerTemplateGenerator(smirnoff.generator)

In [8]:
## Read in PDB
# pdb = PDBFile('epi1846961__4k63_i_model_protein.pdb')

In [9]:
from pdbfixer import PDBFixer
from openmm.app import PDBFile

fixer = PDBFixer("epi1846961__4k63_i_model_protein.pdb")
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingHydrogens()
PDBFile.writeFile(fixer.topology, fixer.positions, open("fixed.pdb", "w"))

pdb = PDBFile("fixed.pdb")

In [10]:
modeller = Modeller(pdb.topology, pdb.positions)

In [11]:
## Make an OpenFF Topology of the ligand
molecule_off_topology = offTopology.from_molecules(molecules=[molecule])

In [12]:
## Convert it to an OpenMM Topology
molecule_topology = molecule_off_topology.to_openmm()

In [13]:
## Get the positions of the ligand
molecule_positions = offquantity_to_openmm(molecule.conformers[0])

In [14]:
## Add the ligand to the Modeller
modeller.add(molecule_topology, molecule_positions)

In [15]:
## Solvate
modeller.addSolvent(forcefield, padding=1.0*nanometer, ionicStrength=0.15*molar)


In [16]:
## Create system
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)

In [17]:
## Setup Simulation
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)

In [ ]:
## Minimize Energy
print("Minimizing Energy...")
simulation.minimizeEnergy()

simulation.reporters.append(PDBReporter('output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, volume=True))
simulation.reporters.append(StateDataReporter("md_log.txt", 100, step=True,
        potentialEnergy=True, temperature=True, volume=True))

Minimizing Energy...


In [ ]:
system.addForce(MonteCarloBarostat(1*bar, 300*kelvin))
simulation.context.reinitialize(preserveState=True)

In [ ]:
print("Running NPT...")
simulation.step(10000)

In [ ]:
print("Running NVT...")
simulation.step(10000)

## Plotting

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## Plot data
data = np.loadtxt("md_log.txt", delimiter=',')

step = data[:,0]
potential_energy = data[:,1]
temperature = data[:,2]
volume = data[:,3]

In [ ]:
## Potential Energy
plt.plot(step, potential_energy)
plt.xlabel("Step")
plt.ylabel("Potential energy (kJ/mol)")
plt.savefig("potential_energy.png")

In [ ]:
## Temperature
plt.plot(step, temperature)
plt.xlabel("Step")
plt.ylabel("Temperature (K)")
plt.savefig("temperature.png")

In [ ]:
## Volume
plt.plot(step, volume)
plt.xlabel("Step")
plt.ylabel("Volume (nm^3)")
plt.savefig("volume.png")